In [6]:
# Import required libraries
import pandas as pd

try:
    # Read the CSV file from the specified path
    # Note: Using raw string (r) to handle Windows file paths
    df = pd.read_csv(r'data/Results.csv')
    print("CSV file successfully loaded!")
    
except FileNotFoundError:
    print("Error: The file was not found. Please check if the path 'E:\\Dropbox\\AI\\Data\\Radiology_Reports\\radiology_reports.csv' is correct.")
except pd.errors.EmptyDataError:
    print("Error: The CSV file is empty.")
except Exception as e:
    print(f"An unexpected error occurred while reading the file: {str(e)}")

print(df.head())


CSV file successfully loaded!
              MRN           EXAM DATE/TIME            VERIFIED DATE  \
0  13995332247820  2021-07-26 12:51:00.000  2021-07-27 17:19:00.000   
1  12633977630399  2021-07-26 11:33:00.000  2021-07-27 17:21:00.000   
2  13164894301526  2021-07-27 21:14:00.000  2021-07-27 21:38:00.000   
3  14601142299512  2022-01-16 09:22:00.000  2022-01-16 09:25:00.000   
4  17207276046856  2023-09-12 14:49:00.000  2023-09-13 06:26:00.000   

       DATE REPORT ENTERED                  VERIFYING PHYSICIAN  \
0  2021-07-27 17:19:00.000            MUJLLI,MOHAMMAD IQAB MOHD   
1  2021-07-27 17:21:00.000  AL KHAWALDEH,KHALED MOHAMMAD GHAL H   
2  2021-07-27 21:38:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   
3  2022-01-16 09:25:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   
4  2023-09-13 06:26:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   

                      TRANSCRIPTIONIST       IMAGING LOCATION  \
0            MUJLLI,MOHAMMAD IQAB MOHD   KHCC 1_MAIN BUILDING   
1  AL KHAW

In [7]:
import pandas as pd
# Read the CSV file
df = pd.read_csv('radiology_results_10.csv')

# Display the column names
print("Column names in the dataset:")
print(df.columns.tolist())



Column names in the dataset:
['MRN', 'EXAM_DATE', 'PROCEDURE', 'REPORT', 'modality', 'body_region', 'comparison_date', 'target_lesions', 'non_target_lesions', 'new_lesions', 'overall_assessment', 'ICDO3_site', 'ICDO3_site_term', 'ICDO3_site_similarity', 'tumor_response', 'error']


In [3]:
import pandas as pd
df= pd.read_csv('data/Results.csv')

# Pull first 10 rows of the data
first_10_rows = df.head(10)

# Print first 10 REPORT entries
print("\nFirst 10 Reports:")
for i, report in enumerate(df['REPORT'].head(10), 1):
    print(f"\nReport #{i}:")
    print(report)
    print("-" * 80)  # Print separator line between reports



First 10 Reports:

Report #1:

Diagnosis: Lymphoma.



Reason: Follow-up.



Comparison: Comparison made to the CT part of PET CT scan dated 2 February 2021.





Findings:



Neck CT with IV contrast:



There are unchanged enlarged hypoattenuating left cervical lymph nodes the large

st seen at levels III measuring 1 cm in short axis.



There are unchanged mildly enlarged right supraclavicular lymph nodes, none exce

eding 1 cm in short axis.



There is no new cervical lymph node enlargement.



Chest CT with IV contrast:



There are unchanged multiple hypoattenuating conglomerate mediastinal lymph node

s are encasing the aortic arch and its major branches with no significant narrow

ing, for example the subcarinal lymph node measures 2 cm in short axis with no s

ignificant change.



There is no new intrathoracic lymph node enlargement.



There is no significant axillary lymph node enlargement.



The mild pericardial thickening/effusion appears less significant.



There is 